# Load Data

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
dfs = pd.read_json('~/Documents/4.MY_PROJECTS/GitHub/predict-used-new-marketplace/data/raw/MLA_100k_checked_v3.jsonlines', lines=True)

In [3]:
dfs = dfs.rename(columns = {'tags':'tag'})
dfs = dfs.rename(columns = {'id':'Id'})

In [9]:
used_count=dfs['title'].str.contains('usado').sum()
if used_count>0:
    print ("There are {m} used".format(m=mel_count))

There are 33 used


## Get features from dictionary columns

In [ ]:
# Get region
dfs['seller_country'] = dfs.apply(lambda x : x['seller_address']['country']['name'], axis = 1)
dfs['seller_state'] = dfs.apply(lambda x : x['seller_address']['state']['name'], axis = 1)
dfs['seller_city'] = dfs.apply(lambda x : x['seller_address']['city']['name'], axis = 1)

In [ ]:
# Transform id (named as descriptions) column to get data
import ast
def str_to_dict(column):
    for i in range(len(column)):
        try:
            column[i] = ast.literal_eval(column[i][0])
        except:
            return

str_to_dict(dfs['descriptions'])

In [ ]:
# get data from descriptions and shipping 
dfs = pd.concat([dfs, dfs["descriptions"].apply(pd.Series)], axis=1)
dfs = pd.concat([dfs, dfs["shipping"].apply(pd.Series)], axis=1)

In [ ]:
pd.set_option('display.max_columns', None)
dfs.head(5)

In [ ]:
# Get payment methods from dict
def convertCol(x,key,i):
    try:
        return x[i][key]
    except: 
        return ''
    
for key in ['description']: #['description','id','type'] -- only description is interesting
    for i in range(0,13):
        dfs[f'payment_{key}{i}'] = dfs['non_mercado_pago_payment_methods'].apply(lambda x: convertCol(x,key,i))

In [ ]:
# Create a boolean column for each payment method 
lista_c = []
for i in range(0,13):
    lista = dfs[f'payment_description{i}'].unique()
    lista_c.extend(lista)

desc_uniques = set(lista_c)
desc_uniques.remove('')
desc_uniques

In [ ]:
# Rename column for an improved dataframe (#TODO: Use apply for performance)
for col in desc_uniques:
    col_name=col.replace(' ','_')
    dfs[col_name] = dfs.isin([col]).any(axis=1)

# drop older columns
dfs = dfs.drop(dfs.loc[:, 'payment_description0':'payment_description12'], axis = 1)

In [ ]:
import numpy as np
dfs = dfs.applymap(lambda x: x if x else np.nan)
dfs = dfs.dropna(how='all', axis=1)

# Data Transformation

## Change type and filter columns

COLUMNS THAT MATTERS:
- warranty (good and new products have different kind of warranties)
- sub_status (when a product ad is suspended might be due it's condition)
- base_price (price are different when used or new)
- seller_id (different sellers might sell used or new items)
- price (price again)
- buying_mode (type of buying might implicate something)
- parent_item_id (might have correlation between similar products)
- last_updated (we'll check)
- id (we'll check)
- official_store_id (different stores sells different items and conditions)
- accepts_mercadopago (new products might accept more payment methods)
- original_price (price again)
- currency_id (type of payment and currency might be due to the kind of seller and products)
- title (keep title to find product)
- automatic_relist (we'll check')
- stop_time (time might influece)
- status (status might influece)
- video_id fica (we'll check, but might be videos for used products)
- initial_quantity (a good feature, used products have low counts)
- start_time (time again)
- sold_quantity (quantity again)
- available_quantity (quantity again)
- seller_country, state, city (used or new ads might have imbalanced distribution between regions)
- local_pick_up (being new or used might influence if local pickup is available)
- free_shipping (big sellers for new products might be more capable of assuming free shipping)
- Contra_reembolso fica (payment methods matters)
- Giro_postal (stays)
- mode fica (don't know what it is but it's full, 'not_specified' might be more common on used products)
- tags (we'll check about tags)
- tag (we'll check about tag)
- date_created
- category 

TRANSFORM COLUMNS (TYPE OF PAYMENTS):
- Cheque_certificado
- Mastercard_Maestro
- Diners
- Transferencia_bancaria
- MercadoPago (será? duplicado com accepts mercadopago?)
- Efectivo
- Tarjeta_de_crédito (duplicado com outras colunas? mesclar colunas e preencher essa)
- American_Express
- MasterCard
- Visa_Electron
- Visa
- Acordar_con_el_comprador

COLUMNS WE DON'T NEED:
- seller_address (too specific)
- deals_ids (nothing relevant, we checked)
- shipping (nothing relevant, we checked)
- non_mercad_pago_etc (transformed)
- site_id (too specific)
- listin_type_id sai
- description (nothing relevant, we checked, turned out to be id)
- international_delivery_mode
- pictures (nothing relevant, we checked)
- thumbnail (nothing relevant, we checked)
- secure_thumbnail (nothing relevant, we checked)
- permalink (nothing relevant, we checked)
- free_methods (nothing relevant, we checked)

DOUBTS:
- variations
- attributes
- dimension

In [ ]:
# Rename columns
dfs = dfs.rename(columns = {'id':'descr_id', 'Id': 'id'})

# Reorder columns
dfs = dfs[['title', 'condition', 'warranty','initial_quantity', 'available_quantity', 'sold_quantity',
                'sub_status', 'buying_mode', 'original_price', 'base_price', 'price', 'currency_id',
                'seller_country', 'seller_state', 'seller_city', 'Giro_postal',  
                'free_shipping', 'local_pick_up', 'mode', 'tags', 'tag',
                'Contra_reembolso','Acordar_con_el_comprador', 'Cheque_certificado', 'Efectivo', 'Transferencia_bancaria', 'Tarjeta_de_crédito',
                'Mastercard_Maestro', 'MasterCard', 'Visa_Electron', 'Visa', 'Diners', 'American_Express',
                'status', 'automatic_relist',
                'accepts_mercadopago', 'MercadoPago', 
                'id', 'descr_id', 'deal_ids', 'parent_item_id', 'category_id', 'seller_id', 'official_store_id', 'video_id',
                'date_created', 'start_time', 'last_updated', 'stop_time']]

In [ ]:
dfs['accepts_mercadopago'].value_counts()

In [ ]:
dfs['MercadoPago'].value_counts()

In [ ]:
# Merge columns about same subjects
dfs['accepts_mercadopago'] = dfs['accepts_mercadopago'].fillna(dfs['MercadoPago'])

In [ ]:
dfs['MasterCard'].value_counts()

In [ ]:
dfs['MasterCard'] = dfs['Mastercard_Maestro'].fillna(dfs['MercadoPago'])

In [ ]:
dfs['Visa'] = dfs['Visa_Electron'].fillna(dfs['Visa'])

In [ ]:
dfs['Tarjeta_de_crédito'].value_counts()

In [ ]:
dfs['Tarjeta_de_crédito'] = dfs['Tarjeta_de_crédito'].fillna(dfs['Visa'])
dfs['Tarjeta_de_crédito'] = dfs['Tarjeta_de_crédito'].fillna(dfs['MasterCard'])
dfs['Tarjeta_de_crédito'] = dfs['Tarjeta_de_crédito'].fillna(dfs['Diners'])
dfs['Tarjeta_de_crédito'] = dfs['Tarjeta_de_crédito'].fillna(dfs['American_Express'])
dfs['Tarjeta_de_crédito'] = dfs['Tarjeta_de_crédito'].fillna(dfs['Visa'])

In [ ]:
dfs['Tarjeta_de_crédito'].value_counts()

In [ ]:
dfs = dfs.rename(columns = {'Tarjeta_de_crédito':'Aceptan_Tarjeta'})

In [ ]:
# Drop used columns
dfs = dfs.drop(columns=['MercadoPago', 'Mastercard_Maestro', 'Visa_Electron'])
dfs = dfs.drop(columns=['Visa', 'MasterCard', 'Diners', 'American_Express'])

In [ ]:
# Treat columns to access data
def try_join(l):
    try:
        return ','.join(map(str, l))
    except TypeError:
        return np.nan

dfs['sub_status'] = try_join(dfs['sub_status'])
dfs['tags'] = try_join(dfs['tags'])

In [ ]:
dfs.columns

In [ ]:
dfs.info()

In [ ]:
# Transform some columns to boolean type
dfs[['Giro_postal', 'free_shipping', 'local_pick_up', 'Contra_reembolso', 
     'Acordar_con_el_comprador', 'Cheque_certificado', 'Efectivo', 
     'Transferencia_bancaria', 'Aceptan_Tarjeta', 'automatic_relist']] = dfs[['Giro_postal', 'free_shipping', 'local_pick_up', 'Contra_reembolso', 
                                                          'Acordar_con_el_comprador', 'Cheque_certificado', 'Efectivo', 
                                                          'Transferencia_bancaria', 'Aceptan_Tarjeta', 'automatic_relist']].notna()

In [ ]:
# Transform type of all columns
dfs = dfs.astype({'title':'str',
                  'condition': 'category', #bool
                  'warranty': 'category',
                  'initial_quantity': 'float', #int
                  'available_quantity': 'float', #int
                  'sold_quantity': 'float', #int
                  'sub_status': 'category', #bool?
                  'buying_mode': 'category',
                  'original_price': 'float',
                  'base_price': 'float',
                  'price': 'float',
                  'currency_id': 'category',
                  'seller_country': 'category',
                  'seller_state': 'category',
                  'seller_city': 'category',
                  'Giro_postal': 'bool',
                  'free_shipping': 'bool',
                  'local_pick_up': 'bool',
                  'mode': 'category',
                  'tags': 'category', #bool?
                  #'tag': 'category',
                  'Contra_reembolso': 'bool',
                  'Acordar_con_el_comprador': 'bool',
                  'Cheque_certificado': 'bool',
                  'Efectivo': 'bool',
                  'Transferencia_bancaria': 'bool',
                  'Aceptan_Tarjeta': 'bool',
                  'id': 'category',
                  'descr_id': 'category',
                  #'deal_ids': 'category',
                  'parent_item_id': 'category',
                  'category_id': 'category',
                  'seller_id': 'category',
                  'official_store_id': 'category',
                  'video_id': 'category',
                  #'date_created': 'datetime',
                  # 'start_time': 'datetime',
                  # 'last_updated': 'datetime',
                  # 'stop_time': 'datetime',
                  'status': 'category', #bool?
                  'automatic_relist': 'bool'
                                         })

In [ ]:
dfs.columns

In [ ]:
# Check missing values
import numpy as np
import pandas as pd

def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table

missing_zero_values_table(dfs)

In [ ]:
display(dfs['seller_country'].value_counts())
dfs = dfs.drop(columns = 'seller_country') # We can drop Country column, it's always Argentina
display(dfs['seller_city'].mode()[0])
display(dfs['seller_state'].mode()[0])
dfs['seller_city'] = dfs['seller_city'].fillna(dfs['seller_city'].mode()[0])
dfs['seller_state'] = dfs['seller_state'].fillna(dfs['seller_state'].mode()[0])

In [ ]:
dfs['accepts_mercadopago'] = dfs['accepts_mercadopago'].fillna(False)
dfs['sold_quantity'] = dfs['sold_quantity'].fillna(0) # Is it ok to fill sold_quantity with 0? [VALIDATE]

In [ ]:
dfs['warranty'] = dfs['warranty'].replace(r'^\s*$', np.nan, regex=True)
dfs['warranty'].isna().sum()

In [ ]:
import pandas as pd
df_temp1 = dfs[dfs['warranty'].isnull()]
df_temp1['warranty'] = False

df_temp2 = dfs[~dfs['warranty'].isnull()]
df_temp2['warranty'] = True

frames = [df_temp1, df_temp2]
dfs = pd.concat(frames)
dfs = dfs.astype({'warranty':'bool'})

In [ ]:
dfs['warranty'].value_counts()

In [ ]:
display('number of sold_quantity', dfs.sold_quantity.nunique())

In [ ]:
def get_value_per_cat():
    flag = dfs.select_dtypes(include=['category']).shape[1]
    i = 0

    while i <= flag:
        print(dict(dfs.select_dtypes(include=['category']).iloc[:,i:i+1].nunique()))
        i = i+1

get_value_per_cat()

In [ ]:
dfs.columns

In [ ]:
import re
dfs['sub_status'] = dfs['sub_status'].str.replace('nan,','')
dfs['sub_status'] = dfs['sub_status'].str.replace(',nan','')
display(len(re.findall(r'suspended',dfs['sub_status'][1])))
display(dfs['sub_status'].value_counts().value_counts())
display(dfs.shape)

# We concluded this column is useless: every row has the same count of the same value ('suspended')
dfs = dfs.drop('sub_status', axis=1)

In [ ]:
# dfs['tags'] = dfs['tags'].str.replace('nan,','')
# dfs['tags'] = dfs['tags'].str.replace(',nan','')

# from ast import literal_eval
# dfs['tags'] = dfs['tags'].apply(lambda x: literal_eval(str(x)))

# def deduplicate(column):
#     flag = len(column)
#     i = 0
    
#     while i <= flag:
#         try:
#             # 1. Convert into list of tuples
#             tpls = [tuple(x) for x in column[i]]
#             # 2. Create dictionary with empty values and
#             # 3. convert back to a list (dups removed)
#             dct = list(dict.fromkeys(tpls))
#             # 4. Convert list of tuples to list of lists
#             dup_free = [list(x) for x in lst]
#             # Print everything
#             column[i] = list(map(''.join, dup_free))
#             # [[1, 1], [0, 1], [0, 1], [1, 1]]
#             i = i+1
#         except:
#             return
        
# deduplicate(dfs['tags'])
# display(dfs['tags'].value_counts().value_counts())
# display(dfs.shape)
# display(dfs['tag'].value_counts().value_counts())

# Other useless colums -- all rows have the same values
dfs = dfs.drop('tags', axis=1)
dfs = dfs.drop('tag', axis=1)        

In [ ]:
display('dataframe shape', dfs.shape)
display('unique ids', dfs.id.nunique())
display('number of sellers', dfs.seller_id.nunique())
display('number of categories', dfs.category_id.nunique())

#Drop useless column
dfs = dfs.drop(['id'], axis=1)

In [ ]:
missing_zero_values_table(dfs)

In [ ]:
dfs = dfs.dropna(axis=1) # drop all columns with missing values (we checked and they are not necessary or have too many missing values to imput properly)

In [ ]:
# Deal with datetimes to create new features
dfs['year_start'] = pd.to_datetime(dfs['start_time']).dt.year.astype('category')
dfs['month_start'] = pd.to_datetime(dfs['start_time']).dt.month.astype('category')
dfs['year_stop'] = pd.to_datetime(dfs['stop_time']).dt.year.astype('category')
dfs['month_stop'] = pd.to_datetime(dfs['stop_time']).dt.month.astype('category')
dfs['week_day'] = pd.to_datetime(dfs['stop_time']).dt.weekday.astype('category')
#dfs['days_active'] = (dfs['start_time'] - dfs['stop_time']).dt.days
dfs['days_active'] = [int(i.days) for i in (dfs.stop_time - dfs.start_time)]
dfs['days_active'] = dfs['days_active'].astype('int')
dfs = dfs.reset_index(drop=True)

#dfs = dfs.drop(['date_created', 'start_time', 'last_updated', 'stop_time'], axis=1)
boxplot = dfs.boxplot(column=['days_active'], showfliers=False)

# Feature Selection

In [ ]:
from sklearn import preprocessing

mylist = list(dfs.select_dtypes(include=['category']).columns)
dfs[mylist] = dfs[mylist].apply(preprocessing.LabelEncoder().fit_transform)

In [ ]:
import seaborn as sns
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]
corr = dfs.corr()
corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

In [ ]:
import numpy as np

columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.9:
            if columns[j]:
                columns[j] = False
                
dfs.columns

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
condition = {'new': 0,'used': 1}

#dfs.condition = [condition[item] for item in dfs.condition]


dfs = dfs.sample(100000).reset_index(drop=True)
#dfs = dfs.reset_index(drop=True)


numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'category', 'bool']

#X = dfs.select_dtypes(include=numerics).drop(columns=['condition'], axis=1)
X = dfs.select_dtypes(include=numerics).drop(columns=['condition'], axis=1)
#X = dfs.drop(columns='condition')
y = dfs.condition

# clf = LogisticRegression().fit(X, y)
# result = permutation_importance(clf, X, y, n_repeats=10,
#                                  random_state=0)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,y)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(13).plot(kind='barh')
plt.show()

In [ ]:
list_1 = feat_importances.nlargest(13)

In [ ]:
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import spearmanr
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform

from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X, y)
print("Accuracy on test data: {:.2f}".format(clf.score(X, y)))

result = permutation_importance(clf, X, y, n_repeats=10, random_state=42, n_jobs=-1) # number of repeats 10
perm_sorted_idx = result.importances_mean.argsort()

tree_importance_sorted_idx = np.argsort(clf.feature_importances_)
tree_indices = np.arange(0, len(clf.feature_importances_)) + 0.5

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
ax1.barh(tree_indices, clf.feature_importances_[tree_importance_sorted_idx], height=0.7)
ax1.set_yticks(tree_indices)
ax1.set_yticklabels(X.columns[tree_importance_sorted_idx])
ax1.set_ylim((0, len(clf.feature_importances_)))
ax2.boxplot(
    result.importances[perm_sorted_idx].T,
    vert=False,
    labels=X.columns[perm_sorted_idx],
)
fig.tight_layout()
plt.show()

In [ ]:
list_2 = reversed(X.columns[tree_importance_sorted_idx])
list_2 = list(list_2)[0:13]

In [ ]:
list_2

In [ ]:
# Feature importance based on mean decrease in impurity
from sklearn.ensemble import RandomForestClassifier

feature_names = [f"feature {i}" for i in range(X.shape[1])]
forest = RandomForestClassifier(random_state=0)
forest.fit(X, y)


import time
import numpy as np

start_time = time.time()
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
elapsed_time = time.time() - start_time

display(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

import pandas as pd

forest_importances = pd.Series(importances, index=X.columns)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
#ax.set_xticks(tree_indices)
#ax.set_xticklabels(X.columns)
fig.tight_layout()

In [ ]:
list_3 = forest_importances.nlargest(13)

In [ ]:
# # Feature importance based on permutation
from sklearn.inspection import permutation_importance

start_time = time.time()
result = permutation_importance(
    forest, X, y, n_repeats=10, random_state=42, n_jobs=-1 # n_repeats = 10 
)
elapsed_time = time.time() - start_time
display(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")



forest_importances = pd.Series(result.importances_mean, index=X.columns)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
ax.set_title("Feature importances using permutation on full model")
ax.set_ylabel("Mean accuracy decrease")
#ax.set_xticks(tree_indices)
#ax.set_xticklabels(X.columns)
fig.tight_layout()
plt.show()

In [ ]:
list_4 = forest_importances.nlargest(13)

In [ ]:
# # Handling multicollinearity
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
# corr = spearmanr(X.dropna(how='any')).correlation

# # Ensure the correlation matrix is symmetric
# corr = (corr + corr.T) / 2
# np.fill_diagonal(corr, 1)

# # We convert the correlation matrix to a distance matrix before performing
# # hierarchical clustering using Ward's linkage.
# distance_matrix = 1 - np.abs(corr)
# dist_linkage = hierarchy.ward(squareform(distance_matrix))
# dendro = hierarchy.dendrogram(
#     dist_linkage, labels=data.feature_names.tolist(), ax=ax1, leaf_rotation=90
# )
# dendro_idx = np.arange(0, len(dendro["ivl"]))

# ax2.imshow(corr[dendro["leaves"], :][:, dendro["leaves"]])
# ax2.set_xticks(dendro_idx)
# ax2.set_yticks(dendro_idx)
# ax2.set_xticklabels(dendro["ivl"], rotation="vertical")
# ax2.set_yticklabels(dendro["ivl"])
# fig.tight_layout()
# plt.show()

In [ ]:
# cluster_ids = hierarchy.fcluster(dist_linkage, 1, criterion="distance")
# cluster_id_to_feature_ids = defaultdict(list)
# for idx, cluster_id in enumerate(cluster_ids):
#     cluster_id_to_feature_ids[cluster_id].append(idx)
# selected_features = [v[0] for v in cluster_id_to_feature_ids.values()]

# X_train_sel = X_train[:, selected_features]
# X_test_sel = X_test[:, selected_features]

# clf_sel = RandomForestClassifier(n_estimators=100, random_state=42)
# clf_sel.fit(X_train_sel, y_train)
# print(
#     "Accuracy on test data with features removed: {:.2f}".format(
#         clf_sel.score(X_test_sel, y_test)
#     )
# )

In [ ]:
# import time
# import matplotlib.pyplot as plt

# start_time = time.time()
# img_shape = X.shape
# importances = forest.feature_importances_
# elapsed_time = time.time() - start_time

# print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")
# imp_reshaped = importances.reshape(img_shape)
# plt.matshow(imp_reshaped, cmap=plt.cm.hot)
# plt.title("Pixel importances using impurity values")
# plt.colorbar()
# plt.show()

In [ ]:
# from sklearn.model_selection import train_test_split

# # Splits the dataset
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# from sklearn.ensemble import RandomForestRegressor
# # Defines the estimator used by the Boruta algorithm
# estimator = RandomForestRegressor()

# from boruta import BorutaPy
# # Creates the BorutaPy object
# boruta = BorutaPy(estimator = estimator, n_estimators = 'auto', max_iter = 100)
# # Fits Boruta
# boruta.fit(np.array(X_train), np.array(y_train))

# Important features
important = list(X.columns[boruta.support_])
print(f"Features confirmed as important: {important}")
# Tentative features
tentative = list(X.columns[boruta.support_weak_])
print(f"Unconfirmed features (tentative): {tentative}")
# Unimportant features
unimportant = list(X.columns[~(boruta.support_ | boruta.support_weak_)])
print(f"Features confirmed as unimportant: {unimportant}")

In [ ]:
list_5 = important + tentative

In [ ]:
selected_features = set([y for x in [list_1.index, list_2, list_3.index, list_4.index, list_5] for y in x])
selected_features

In [ ]:
# from BorutaShap import BorutaShap
# import tqdm as notebook_tqdm

# # Creates a BorutaShap selector for regression
# selector = BorutaShap(importance_measure = 'shap', classification = False)

# # Fits the selector
# selector.fit(X = X_train, y = y_train, n_trials = 100, sample = False, verbose = True)
# # n_trials -> number of iterations for Boruta algorithm
# # sample -> samples the data so it goes faster

# # Display features to be removed
# features_to_remove = selector.features_to_remove
# print(features_to_remove)
# # Removes them
# X_train_boruta_shap = X_train.drop(columns = features_to_remove)
# X_test_boruta_shap = X_test.drop(columns = features_to_remove)

In [ ]:
# open file in write mode
with open(r'selected_features.txt', 'w') as fp:
    for item in selected_features:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')